In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import geometry
from quetzal.model import stepmodel
from quetzal.engine import engine, connectivity

In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none func

# Preparation of the transport network.
## Saves access and egress links for each zone.
## Needs all networks.


In [2]:
input_path = '../input/'
output_path = '../output/'
model_path = '../model/'

In [3]:
sm = stepmodel.read_json(model_path + 'de_pt_network')
bus = stepmodel.read_json(model_path + 'de_pt_network_bus')
road = stepmodel.read_json(model_path + 'de_road_network')

### Clean zones

In [4]:
# Drop unused zones for later steps to work 
print(len(sm.zones.index))
sm.zones = sm.zones.loc[sm.zones['NUTS_ID'].str.startswith('DE')]
sm.zones = sm.zones[sm.zones["LEVL_CODE"]==3]
print(len(sm.zones.index))

467
467


In [5]:
# Compute controids
sm.preparation_ntlegs(
    zone_to_transit=False,
    zone_to_road=False)

## Assumptions

In [6]:
# Walking vs. driving to the train station
threshold = 500 # in m
# Average speeds from MiD calibration data
speed_non_motorised_modes = 17 # in km/h
speed_car = 44 # in km/h
speed_footpaths = 5 # in km/h, assumed
# Max. distances for accessing the transport nodes
# in m
threshold_dict = {'road': 10000,
                  'road_pt': 500,
                  'pt': 12000,
                  'footpaths': 500,
                  'footpaths_air': 2200,
                  'cycling_zones': 20000}

## Generate footpaths table

In [7]:
sm.nodes.shape

(14990, 4)

In [8]:
# The Voronoi algorithm is too slow for this amount of nodes
#sm.footpaths = connectivity.build_footpaths(
#    sm.nodes,
#    speed=speed_footpaths,
#    max_length=threshold_dict['footpaths'],
#    n_clusters=1000,
#    coordinates_unit=sm.coordinates_unit)

In [9]:
# Generate footpaths between modes
sm.footpaths = sm.footpaths.iloc[0:0]
for o in ['rail_short_distance']:
    for d in [t for t in sm.pt_route_types if t!='bus']:
        if o != d:
            ntlegs = engine.ntlegs_from_centroids_and_nodes(
                sm.nodes.loc[sm.nodes['route_type']==o],
                sm.nodes.loc[sm.nodes['route_type']==d],
                short_leg_speed=speed_footpaths,
                long_leg_speed=speed_non_motorised_modes,
                threshold=threshold,
                n_neighbors=1,
                coordinates_unit=sm.coordinates_unit)
            if d == 'air':
                ntlegs = ntlegs.loc[ntlegs['distance']<=
                                    threshold_dict['footpaths_air']]
            else:
                ntlegs = ntlegs.loc[ntlegs['distance']<=
                                    threshold_dict['footpaths']]
            sm.footpaths = sm.footpaths.append(ntlegs)
len(sm.footpaths.index)

6632

In [10]:
# Add bus connections
for d in [t for t in sm.pt_route_types if t!='bus']:
    ntlegs = engine.ntlegs_from_centroids_and_nodes(
        bus.nodes,
        sm.nodes.loc[sm.nodes['route_type']==d],
        short_leg_speed=speed_footpaths,
        long_leg_speed=speed_non_motorised_modes,
        threshold=threshold,
        n_neighbors=1,
        coordinates_unit=sm.coordinates_unit)
    if d == 'air':
        ntlegs = ntlegs.loc[ntlegs['distance']<=
                            threshold_dict['footpaths_air']]
    else:
        ntlegs = ntlegs.loc[ntlegs['distance']<=
                            threshold_dict['footpaths']]
    sm.footpaths = sm.footpaths.append(ntlegs)
len(sm.footpaths.index)

104830

In [11]:
# Number of nodes that overlay each other
sm.footpaths.loc[sm.footpaths['distance']==0].shape

(3700, 11)

In [12]:
# Keep only one footpath to stops of the same trip
# List of footpath nodes
node_list = list(sm.footpaths[['a', 'b']].stack())
n_foot_dict = sm.footpaths[['a', 'b']].stack().value_counts().to_dict()
# Reduce list by keeping the most connected stops of each trip
links = sm.links.append(bus.links)
links = links.loc[
    (links['a'].isin(node_list)) | (links['b'].isin(node_list))]
links['n_foot'] = links['a'].map(n_foot_dict) + links['b'].map(n_foot_dict)
node_list = set(list(links.sort_values('n_foot').drop_duplicates(
    'trip_id')[['a', 'b']].stack()))

In [13]:
len(node_list)

56521

In [14]:
# Restrict footpaths to one to two connections between stops of every two trip_ids
sm.footpaths = sm.footpaths.loc[(sm.footpaths['a'].isin(node_list)) |
                                (sm.footpaths['b'].isin(node_list))]
sm.footpaths.shape

(97232, 11)

In [15]:
# Generate footpaths between centroids
ntlegs = engine.ntlegs_from_centroids_and_nodes(
    sm.centroids,
    sm.centroids,
    short_leg_speed=speed_non_motorised_modes,
    long_leg_speed=speed_non_motorised_modes,
    threshold=threshold,
    n_neighbors=2,
    coordinates_unit=sm.coordinates_unit)
ntlegs = ntlegs.loc[ntlegs['distance']<=
                    threshold_dict['cycling_zones']]
ntlegs = ntlegs.loc[ntlegs['distance']!=0]
ntlegs.drop_duplicates(['direction', 'distance', 'time'], inplace=True)
sm.footpaths = sm.footpaths.append(ntlegs)
ntlegs.shape

(338, 11)

In [16]:
# Reindex
sm.footpaths.reset_index(drop=True, inplace=True)
sm.footpaths.index = 'foot_' + pd.Series(sm.footpaths.index).astype(str)

In [17]:
sm.footpaths.sample(n=3)

,a,b,direction,distance,geometry,long_leg_speed,rank,short_leg_speed,speed,speed_factor,time
foot_1062,rail_short_node_7421,rail_long_node_190,access,111.713867,"LINESTRING (13.24843 52.75436, 13.24961 52.75365)",17.0,0.0,5.0,5.0,0.472681,80.433985
foot_65473,rail_short_node_10042,bus_n_38271,eggress,350.366201,"LINESTRING (7.02101 52.21208, 7.02244 52.21511)",17.0,0.0,5.0,5.0,0.837098,252.263665
foot_68644,rail_short_node_13786,bus_n_360682,eggress,76.042036,"LINESTRING (12.80869 49.16356, 12.80792 49.16403)",17.0,0.0,5.0,5.0,0.389980,54.750266


## Add access and egress links from zone centroids

In [18]:
# When having centroids computed as geometric centers of
# political zones, it can happen that centroids come very
# close to each other. This requires special filtering
# of connectors for not making routes unrealistically short
def filter_connectors(df, keep=2, length_weight=None, dist_centr_weight=None,
                      conn_weight=None, conn_dict=None):
    # Each link exists as both access and egress
    ac = df.loc[df['direction']=='access']
    eg = df.loc[df['direction']=='eggress']
    analysis_cols = []
    if length_weight:
        # Link length (in km) weighted
        ac['length'] = - np.power(ac['distance'] / 1e3, length_weight)
        eg['length'] = - np.power(eg['distance'] / 1e3, length_weight)
        analysis_cols.append('length')
    if dist_centr_weight:
        # Calculate distance to the nearest centroid (except own one)
        ac['centr_dist'] = np.power([min([p.distance(c) for c in list(
            sm.centroids.loc[sm.centroids['FID']!=own_c, 'geometry'])]
                                        ) / 1.4e-2 for own_c, p in zip(
            list(ac['a']), [geometry.Point(l.coords[-1]) for l in ac['geometry']])],
                                    dist_centr_weight)
        eg['centr_dist'] = np.power([min([p.distance(c) for c in list(
            sm.centroids.loc[sm.centroids['FID']!=own_c, 'geometry'])]
                                        ) / 1.4e-2 for own_c, p in zip(
            list(eg['b']), [geometry.Point(l.coords[0]) for l in eg['geometry']])],
                                    dist_centr_weight)
        analysis_cols.append('centr_dist')
    if conn_weight:
        # Calculate connectivity
        ac['connectivity'] = np.power(ac['b'].map(n_links_dict), conn_weight)
        eg['connectivity'] = np.power(eg['a'].map(n_links_dict), conn_weight)
        analysis_cols.append('connectivity')
    # Calculate link performance with given attributes
    ac['sum'] = ac[analysis_cols].sum(axis=1)
    eg['sum'] = eg[analysis_cols].sum(axis=1)
    # Get the n most wanted links
    ac = ac.sort_values('sum', ascending=False).groupby('a').head(keep)
    eg = eg.sort_values('sum', ascending=False).groupby('b').head(keep)
    return ac.append(eg).reset_index()[df.columns]

In [19]:
# Compute road access and egress links
sm.zone_to_road = engine.ntlegs_from_centroids_and_nodes(
    sm.centroids,
    road.road_nodes,
    short_leg_speed=speed_footpaths,
    long_leg_speed=speed_non_motorised_modes,
    threshold=threshold,
    n_neighbors=20,
    coordinates_unit=sm.coordinates_unit)
sm.zone_to_road = sm.zone_to_road.loc[
    sm.zone_to_road['distance']<=threshold_dict['road']]
len(sm.zone_to_road.index)

18238

In [20]:
sm.zone_to_road = filter_connectors(sm.zone_to_road, keep=1,
                                    length_weight=None,
                                    dist_centr_weight=1)
len(sm.zone_to_road.index)

932

In [21]:
sm.zones.loc[~sm.zones['FID'].isin(list(sm.zone_to_road['a']))]

,CNTR_CODE,NUTS_NAME,LEVL_CODE,FID,NUTS_ID,population,area,urbanisation,lau_id,geometry
index,,,,,,,,,,
DEF07_2,DE,Nordfriesland 2,3,DEF07_2,DEF07,18317,100.145861,3.0,1054,"POLYGON ((8.57200 54.88428, 8.56424 54.88393, ..."


In [22]:
# Compute road - PT links
sm.road_to_transit = engine.ntlegs_from_centroids_and_nodes(
    sm.nodes,
    road.road_nodes,
    short_leg_speed=speed_footpaths,
    long_leg_speed=speed_non_motorised_modes,
    threshold=threshold,
    n_neighbors=1,
    coordinates_unit=sm.coordinates_unit)
sm.road_to_transit = sm.road_to_transit.loc[
    sm.road_to_transit['distance']<=threshold_dict['road_pt']]
len(sm.road_to_transit.index)

15584

In [23]:
# Every route type is saved once in this attribute
assert len(sm.pt_route_types) == len(sm.links['route_type'].unique())
sm.pt_route_types

['rail_long_distance', 'rail_short_distance', 'coach', 'air']

In [24]:
# How many links does every node have?
n_links_dict = sm.links[['a', 'b']].append(bus.links[['a', 'b']]).append(
    sm.footpaths[['a', 'b']]).stack().value_counts().to_dict()

In [25]:
# Compute PT access and egress links by route type
sm.zone_to_transit = sm.zone_to_transit.iloc[0:0]
for t in sm.pt_route_types:
    ntlegs = engine.ntlegs_from_centroids_and_nodes(
        sm.centroids,
        sm.nodes.loc[sm.nodes['route_type']==t],
        short_leg_speed=speed_footpaths,
        long_leg_speed=speed_non_motorised_modes,
        threshold=threshold,
        n_neighbors=5,
        coordinates_unit=sm.coordinates_unit)
    # Cut off long links
    ntlegs = ntlegs.loc[ntlegs['distance']<=
                        threshold_dict['pt']]
    ntlegs['route_type'] = t
    sm.zone_to_transit = sm.zone_to_transit.append(filter_connectors(
        ntlegs, keep=1, length_weight=1, dist_centr_weight=2,
        conn_weight=.5, conn_dict=n_links_dict))
sm.zone_to_transit.reset_index(drop=True, inplace=True)
len(sm.zone_to_transit.index)

1846

In [26]:
# Add bus connections
ntlegs = engine.ntlegs_from_centroids_and_nodes(
    sm.centroids,
    bus.nodes,
    short_leg_speed=speed_footpaths,
    long_leg_speed=speed_non_motorised_modes,
    threshold=threshold,
    n_neighbors=5, # Generate a bunch and clean later
    coordinates_unit=sm.coordinates_unit)
# Cut off long links
ntlegs = ntlegs.loc[ntlegs['distance']<=
                    threshold_dict['pt']]
ntlegs['route_type'] = 'bus'
len(ntlegs.index)

4652

In [27]:
sm.zone_to_transit = sm.zone_to_transit.append(filter_connectors(
    ntlegs, keep=1, length_weight=1, dist_centr_weight=2,
    conn_weight=.5, conn_dict=n_links_dict))
sm.zone_to_transit.reset_index(drop=True, inplace=True)
len(sm.zone_to_transit.index)

2778

In [28]:
# Number of legs by route type
sm.zone_to_transit.groupby(['route_type', 'direction'])['a'].count()

route_type           direction
air                  access        27
                     eggress       27
bus                  access       466
                     eggress      466
coach                access       241
                     eggress      241
rail_long_distance   access       202
                     eggress      202
rail_short_distance  access       453
                     eggress      453
Name: a, dtype: int64

In [29]:
# Every zone must have an access and an egress link to PT
#sm.zones.loc[sm.zones['FID'].isin(list(sm.zone_to_transit['a']))==False]
assert sm.zones['FID'].isin(list(sm.zone_to_transit['a'])).all()
assert sm.zones['FID'].isin(list(sm.zone_to_transit['b'])).all()

### Parametrise access and egress links
Only zone-PT connectors will be handled in a seperate step

In [30]:
# Road - PT connectors
sm.road_to_transit['distance'] = 0
sm.road_to_transit['time'] = 5*60 # in seconds
sm.road_to_transit.sample(n=2)

,a,b,direction,distance,geometry,rank,speed_factor,short_leg_speed,long_leg_speed,speed,time
15399,1016581884,rail_long_node_806,eggress,0,"LINESTRING (8.813834 53.08348, 8.8142817 53.08...",0,0.662769,5,17,5.0,300
7237,rail_short_node_9579,8402879880,access,0,"LINESTRING (12.172836 53.8006, 12.1723647 53.7...",0,0.918741,5,17,5.0,300


In [31]:
# Road - centroid connectors
sm.zone_to_road['distance'] = 0
sm.zone_to_road['time'] = 0
sm.zone_to_road.sample(n=2)

,a,b,direction,distance,geometry,rank,speed_factor,short_leg_speed,long_leg_speed,speed,time
90,DE918,2947094557,access,0,LINESTRING (9.847955045776768 51.7426266240809...,12,3.431438,5,17,17.000000,0
145,DE137,1993022069,access,0,LINESTRING (8.794348494480698 48.0112772596507...,11,2.247207,5,17,11.236034,0


In [32]:
# Parameterisation comes later for PT
sm.zone_to_transit.sample(n=2)

,a,b,direction,distance,geometry,long_leg_speed,rank,route_type,short_leg_speed,speed,speed_factor,time
1619,coach_node_FLIXBUS:36,DE600,eggress,454.492932,"LINESTRING (10.01186 53.54768, 10.01166 53.55177)",17.0,0.0,coach,5.0,5.0,0.953408,327.234911
38,DE224,rail_long_node_802,access,10017.995201,"LINESTRING (13.00065 48.77867, 12.86394 48.77962)",17.0,0.0,rail_long_distance,5.0,17.0,4.476158,2121.457807


## Save model

In [33]:
# Drop unneccessary columns
cols = ['speed_factor', 'short_leg_speed', 'long_leg_speed', 'rank']
sm.footpaths.drop(cols, axis=1, inplace=True)
sm.zone_to_transit.drop(cols, axis=1, inplace=True)
sm.zone_to_road.drop(cols, axis=1, inplace=True)
sm.road_to_transit.drop(cols, axis=1, inplace=True)
sm.zone_to_transit.drop(['n_links'], axis=1, inplace=True, errors='ignore')

In [34]:
# Make tables lighter
cols = ['distance', 'speed', 'time']
sm.footpaths[cols] = sm.footpaths[cols].astype(int)
sm.zone_to_transit[cols] = sm.zone_to_transit[cols].astype(int)
sm.zone_to_road[cols] = sm.zone_to_road[cols].astype(int)
sm.road_to_transit[cols] = sm.road_to_transit[cols].astype(int)

In [35]:
sm.footpaths.sample()

,a,b,direction,distance,geometry,speed,time
foot_70855,rail_short_node_4911,bus_n_177899,eggress,453,"LINESTRING (7.01732 51.72119, 7.01158 51.71919)",5,326


In [36]:
# Saving model...
tables = ['centroids', 'footpaths', 'zone_to_transit']
sm.to_json(model_path + 'de_pt_access_egress',
           only_attributes=tables, encoding='utf-8')
sm.to_json(model_path + 'de_road_access_egress',
           only_attributes=['centroids', 'zone_to_road', 'road_to_transit'],
           encoding='utf-8')

to_hdf(overwriting): 100%|█████████████████████████████████████████████████████████████| 36/36 [00:02<00:00, 13.96it/s]
